In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestRegressor
df = pd.read_csv('../../Data/complete_fire.csv')
df.head()

In [ ]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
categorical = ['object']
numdf = df_clean.copy().select_dtypes(include=numerics)
catdf = df_clean.copy().select_dtypes(include=categorical)
catdf.drop(columns=['unique_fire_identifier','incident_name'],inplace=True)
catdf.head()

In [ ]:
# will dummify these features to obtain correlations  
dummified_1 = pd.get_dummies(data=catdf,drop_first=False) 
dummified_1 = pd.concat([numdf,dummified_1],axis=1)
dummified_1.head()

In [ ]:
fire_class = []
for obs in df['acres_burned']:
    if 0<=obs <= 0.26:
        fire_class.append('A')
    elif 0.26 <= obs <10.0:
        fire_class.append('B')
    elif 10.0<=obs<100.0:
        fire_class.append('C')
    elif 100.0<=obs<300:
        fire_class.append('D')
    elif 300.0 <= obs <1_000:
        fire_class.append('E')
    elif 1_000.0<=obs<5_000.0:
        fire_class.append('F')
    elif 5_000.0 <= obs <10_000:
        fire_class.append('G')
    elif 10_000.0 <=obs<50_000:
        fire_class.append('H')
    elif 50_000.0<=obs<100_000:
        fire_class.append('I')
    elif 100_000.0<=obs<500_000.0:
        fire_class.append('J')
    elif 500_000.0 <= obs <1_000_000:
        fire_class.append('K')
    elif obs >= 1_000_000.0:
        fire_class.append('L')
    else:
        print(obs)
dummified_1['fire_class'] = fire_class 

In [ ]:
model_1 = dummified_1.drop(columns='acres_burned')
model_1.sort_values(by=['year','month']).head()

In [ ]:
X =mode1_1.drop(columns = 'fire_class') 
y= mode1_1['fire_class']
Xtrain, Xtest, ytrain, ytest = train_test_split(model_1.iloc[:, :500], y, test_size=0.2,shuffle=False)

#random_model = RandomForestRegressor(n_estimators=300, random_state = 42, n_jobs = -1)
